In [ ]:
import socket
import cv2 
import pickle
import struct
import threading

# create socket
server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

HOST     = "192.168.1.4"
PORT   = 20001

socket_address = (HOST,PORT)

# Socket Bind
server_socket.bind(socket_address)

# Socket Listen
server_socket.listen(5)
print("LISTENING AT:",socket_address)

client_sockets = []

def live_stream(): 
    cap = cv2.VideoCapture(0)
    try:
        while(cap.isOpened()):
            ret, frame = cap.read()
            # face detection
            model = cv2.CascadeClassifier('C:\\Users\\mtabishk\\Desktop\\LW-ARTH-ws\\python-live-streaming-with-machine-learning\\tcp-server-client\\haarcascade_frontalface_default.xml')
            faces = model.detectMultiScale(frame)
            if (len(faces) != 0):
                for face in faces:
                    x1 = face[0]
                    y1 = face[1]
                    x2 = x1 + face[2]
                    y2 = y1 + face[3]
                    frame = cv2.rectangle(frame, (x1,y1), (x2,y2), [0,255,0], 3)
            frame = cv2.putText(frame, f"Faces Detected: {len(faces)}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # serialize frame to byte data
            a = pickle.dumps(frame)
            # Q: unsigned long long
            message = struct.pack("Q", len(a) ) + a
            
            for client_socket in client_sockets:
                client_socket.sendall(message)
                
            cv2.imshow('TRANSMITTING VIDEO',frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(e)
    finally:
        cap.release()
        cv2.destroyAllWindows()
        for client_socket in client_sockets:
            print(client_socket)
            client_socket.close()
            client_sockets.pop()
        
while True:
    client_socket,addr = server_socket.accept()
    print('GOT CONNECTION FROM:',addr)
    
    client_sockets.append(client_socket)
    if len(client_sockets) == 1:
        t1 = threading.Thread(target=live_stream)
        t1.start()
    
    